# 根据人工发现的规律去查找合适的聚类数目

## 根据人工经验找到有问题号码

In [1]:
from sklearn.preprocessing import scale
import pandas as pd
import numpy as np

In [36]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns',100)
df=pd.read_csv('../data/20180105_result.csv',encoding='gbk')
df.shape

(316827, 10)

In [37]:
df["TAG"] = 0
def test(x):
    if x["NUM_CALL"] >= 10:   # 总拨打次数大于20
        x["TAG"] = 1    #   125条
    elif x["NUM_CALL"] >= 4 and x["MOST_DURATION"] <= 6 and x["MOST_DURATION_NUM"] >= 3:
        #   总拨打次数大于等于4  通话某个时长数最多的数等于6或0   通话时长数最多的次数大于等于3
        #        5    970
        #        4    986
        x["TAG"] = 1    #   262条
    return x
df1 = df.apply(test, axis=1)

筛选出有问题号码

In [40]:
err_num = df1[df1['TAG'] == 1]['CALLING_NUM']

In [45]:
err_num = err_num.tolist()

In [73]:
total = len(err_num)

## 定义比对函数方便后续使用, 百分比在70%-100%之间的聚类就归为好的聚类

In [223]:
def compare_label(err_num, cluster_num, loop_num):
    bad_num = 0
    bad_percent_list = []
    for i in range(loop_num):
        hint = 0
        item_calling_num = cluster_num[cluster_num['label'] ==  i]['CALLING_NUM']
        item_calling_num_size = item_calling_num.size
        if not item_calling_num_size:
            continue
        for calling_num in item_calling_num:
            if calling_num in err_num:
                hint += 1
        if hint / float(item_calling_num_size) > 0.7:
            bad_num += hint
        # print("------------cluster:{0}, bad_num:{1}, bad_percent:{2}, total:{3}----------".format(i, hint,  hint / float(item_calling_num_size), item_calling_num_size))
    print('bad_percent is : {0}'.format(bad_num / float(total))) #>70%的算在聚类成功

In [224]:
from sklearn.cluster import KMeans

In [225]:
from sklearn.cluster import MiniBatchKMeans

In [392]:
def cluster_kmeans(data, fit_data):
    for i in range(70, 105, 5):
        print('------------------------------cluster:{0}-----------------------'.format(i))
        # model = MiniBatchKMeans(n_clusters=i, init='k-means++') # 指定聚类数目i
        model = KMeans(n_clusters=i, init='k-means++', n_init=30, n_jobs=100) # 指定聚类数目i
        y_pred = model.fit_predict(pca_data)
        cluster_data = data.copy()
        cluster_data['label'] = y_pred
        compare_label(err_num, cluster_data, i)

# 各种数据处理后用k-means聚类 <br\>
<div style="font-size:20px"> 结合来看，比较好的分类为75类<br\>
    而且即使最好的分类，每一类中可疑号码占所属类的70%以上的总<br\>
    的可疑号码占根据规律发现的总的可疑号码也只能达到50%而已。

</div> 

* <div style="font-size:24px"> 利用pca降维后再聚类</div> 

In [393]:
from sklearn.decomposition import PCA

In [394]:
pca = PCA(n_components=3) # 指定降维到多少

In [395]:
data = pd.read_csv("../data/20180105_scale.csv")

In [396]:
pca_data = data.copy()

去除电话号码

In [397]:
calling_num = pca_data.pop('CALLING_NUM')

查看降维后的分布

In [398]:
pca.fit(pca_data)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [399]:
pca.explained_variance_ratio_

array([ 0.35580874,  0.25856489,  0.12593122])

In [400]:
pca.explained_variance_

array([ 3.20228875,  2.32709134,  1.13338455])

In [401]:
pca_data = pca.transform(pca_data)

In [425]:
cluster_kmeans(data, pca_data)

------------------------------cluster:70-----------------------
bad_percent is : 0.507099391481
------------------------------cluster:75-----------------------
bad_percent is : 0.507099391481
------------------------------cluster:80-----------------------
bad_percent is : 0.312373225152
------------------------------cluster:85-----------------------
bad_percent is : 0.233265720081
------------------------------cluster:90-----------------------
bad_percent is : 0.424949290061
------------------------------cluster:95-----------------------
bad_percent is : 0.446247464503
------------------------------cluster:100-----------------------
bad_percent is : 0.384381338742


* <div style="font-size:24px">不用PCA降维</div> 

In [426]:
normal_data = data.copy()

In [427]:
calling_num = normal_data.pop('CALLING_NUM')

In [428]:
cluster_kmeans(data, normal_data)

------------------------------cluster:70-----------------------
bad_percent is : 0.276876267748
------------------------------cluster:75-----------------------
bad_percent is : 0.338742393509
------------------------------cluster:80-----------------------
bad_percent is : 0.460446247465
------------------------------cluster:85-----------------------
bad_percent is : 0.463488843813
------------------------------cluster:90-----------------------
bad_percent is : 0.430020283976
------------------------------cluster:95-----------------------
bad_percent is : 0.415821501014
------------------------------cluster:100-----------------------
bad_percent is : 0.337728194726


* <div style="font-size:24px">利用原始数据（../data/20180105_result.csv）数据标准化后在聚类</div> 

In [429]:
result_data = pd.read_csv("../data/20180105_result.csv")

In [430]:
result_data2 = result_data.copy()

In [431]:
calling_num = result_data2.pop('CALLING_NUM')

In [432]:
data_zs = 1.0 * (result_data2 - result_data2.mean()) / result_data2.std() #数据标准化

In [433]:
cluster_kmeans(result_data, data_zs)

------------------------------cluster:70-----------------------
bad_percent is : 0.393509127789
------------------------------cluster:75-----------------------
bad_percent is : 0.430020283976
------------------------------cluster:80-----------------------
bad_percent is : 0.347870182556
------------------------------cluster:85-----------------------
bad_percent is : 0.401622718053
------------------------------cluster:90-----------------------
bad_percent is : 0.447261663286
------------------------------cluster:95-----------------------
bad_percent is : 0.393509127789
------------------------------cluster:100-----------------------
bad_percent is : 0.222109533469


* <div style="font-size:24px">利用原始数据（../data/20180105_result.csv）聚类</div> 

In [434]:
result_data3 = result_data.copy()

In [435]:
calling_num = result_data3.pop('CALLING_NUM')

In [436]:
cluster_kmeans(result_data, result_data3)

------------------------------cluster:70-----------------------
bad_percent is : 0.378296146045
------------------------------cluster:75-----------------------
bad_percent is : 0.465517241379
------------------------------cluster:80-----------------------
bad_percent is : 0.367139959432
------------------------------cluster:85-----------------------
bad_percent is : 0.429006085193
------------------------------cluster:90-----------------------
bad_percent is : 0.463488843813
------------------------------cluster:95-----------------------
bad_percent is : 0.409736308316
------------------------------cluster:100-----------------------
bad_percent is : 0.321501014199


* <div style="font-size:24px">利用原始数据（../data/20180105_result.csv）包含电话号码聚类</div> 

In [437]:
result_data4 = result_data.copy()

In [438]:
cluster_kmeans(result_data, result_data4)

------------------------------cluster:70-----------------------
bad_percent is : 0.313387423935
------------------------------cluster:75-----------------------
bad_percent is : 0.409736308316
------------------------------cluster:80-----------------------
bad_percent is : 0.364097363083
------------------------------cluster:85-----------------------
bad_percent is : 0.502028397566
------------------------------cluster:90-----------------------
bad_percent is : 0.227180527383
------------------------------cluster:95-----------------------
bad_percent is : 0.431034482759
------------------------------cluster:100-----------------------
bad_percent is : 0.43509127789


* <div style="font-size:24px">利用原始数据（../data/20180105_result.csv）PCA降维，包含电话号码聚类</div> 

In [439]:
result_data5 = result_data.copy()

In [440]:
from sklearn.preprocessing import StandardScaler

In [441]:
result_data5 = StandardScaler().fit_transform(result_data5)

In [442]:
pca = PCA(n_components=3) # 指定降维到多少

In [443]:
pca.fit(result_data5)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [444]:
pca.explained_variance_ratio_

array([ 0.32025342,  0.23271692,  0.1133381 ])

In [445]:
pca.explained_variance_

array([ 3.20254428,  2.32717654,  1.13338462])

In [446]:
pca_data2 = pca.transform(result_data5)

In [447]:
cluster_kmeans(result_data, pca_data2)

------------------------------cluster:70-----------------------
bad_percent is : 0.352941176471
------------------------------cluster:75-----------------------
bad_percent is : 0.424949290061
------------------------------cluster:80-----------------------
bad_percent is : 0.419878296146
------------------------------cluster:85-----------------------
bad_percent is : 0.358012170385
------------------------------cluster:90-----------------------
bad_percent is : 0.407707910751
------------------------------cluster:95-----------------------
bad_percent is : 0.470588235294
------------------------------cluster:100-----------------------
bad_percent is : 0.44523326572
